In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("C:\\Users\\ssc\\Documents\\retail_store_sales.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


# Part 1: Data Cleaning & Preparation

## 1. Missing values

In [5]:
# Which columns have missing values?
df.isnull().sum()

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64

In [6]:
# What percentage of the dataset is missing in each column?
print((df.isnull().mean() * 100))

Transaction ID       0.000000
Customer ID          0.000000
Category             0.000000
Item                 9.646123
Price Per Unit       4.842942
Quantity             4.803181
Total Spent          4.803181
Payment Method       0.000000
Location             0.000000
Transaction Date     0.000000
Discount Applied    33.391650
dtype: float64


In [7]:
# Suggest and apply strategies to fill/fix missing values for:
# 	Item
# 	Quantity
# 	Total Spent
# 	Discount Applied

df = df.fillna({col: df[col].median() if df[col].dtype != 'object' else df[col].mode().iloc[0] for col in df.columns})
df.sample(10)

C:\Users\ssc\AppData\Local\Temp\ipykernel_8488\996971064.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna({col: df[col].median() if df[col].dtype != 'object' else df[col].mode().iloc[0] for col in df.columns})


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
2821,TXN_3595305,CUST_03,Computers and electric accessories,Item_19_CEA,32.0,1.0,32.0,Cash,In-store,2023-04-16,True
1226,TXN_5012256,CUST_10,Furniture,Item_24_FUR,39.5,4.0,158.0,Digital Wallet,In-store,2022-04-16,False
11021,TXN_4235196,CUST_13,Food,Item_6_FOOD,12.5,9.0,112.5,Digital Wallet,In-store,2024-02-28,False
744,TXN_9906990,CUST_22,Butchers,Item_3_BUT,8.0,9.0,72.0,Digital Wallet,In-store,2022-07-11,False
11445,TXN_3939276,CUST_18,Furniture,Item_25_FUR,41.0,2.0,82.0,Cash,In-store,2022-05-04,True
2928,TXN_3545596,CUST_10,Beverages,Item_10_BEV,18.5,7.0,129.5,Credit Card,In-store,2023-03-26,True
444,TXN_5809744,CUST_18,Food,Item_9_FOOD,17.0,4.0,68.0,Cash,In-store,2023-01-29,True
6989,TXN_8662660,CUST_14,Furniture,Item_2_FUR,6.5,10.0,65.0,Credit Card,Online,2022-05-29,True
3622,TXN_9216790,CUST_15,Computers and electric accessories,Item_2_CEA,6.5,2.0,13.0,Credit Card,Online,2024-09-06,True
10939,TXN_3314885,CUST_23,Electric household essentials,Item_25_EHE,41.0,2.0,82.0,Credit Card,Online,2024-01-17,True


In [8]:
df.isnull().sum()

Transaction ID      0
Customer ID         0
Category            0
Item                0
Price Per Unit      0
Quantity            0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
Discount Applied    0
dtype: int64

## 2.Data consistency

In [9]:
# Are all Transaction IDs unique? If not, identify duplicates.

# Check if all Transaction IDs are unique:
print(df['Transaction ID'].is_unique)

# Find duplicate Transaction IDs:
print("No of duplicated of transaction ", df[df['Transaction ID'].duplicated(keep=False)])

# Count of duplicate Transaction IDs:
print("No of duplicated values",df['Transaction ID'].duplicated().sum())


True
No of duplicated of transaction  Empty DataFrame
Columns: [Transaction ID, Customer ID, Category, Item, Price Per Unit, Quantity, Total Spent, Payment Method, Location, Transaction Date, Discount Applied]
Index: []
No of duplicated values 0


In [10]:
# Are there duplicate rows in the dataset?
df.duplicated().sum()

np.int64(0)

In [11]:
# 	Verify whether Price Per Unit × Quantity = Total Spent. Flag mismatches.
df['mismatch'] = df['Price Per Unit'] * df['Quantity'] != df['Total Spent']
df[df['mismatch']].sample(3)

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,mismatch
1924,TXN_8084251,CUST_16,Electric household essentials,Item_2_BEV,26.0,6.0,108.5,Digital Wallet,In-store,2024-01-03,True,True
3024,TXN_2792485,CUST_04,Computers and electric accessories,Item_2_BEV,6.5,6.0,108.5,Digital Wallet,In-store,2022-08-09,True,True
2153,TXN_5338814,CUST_06,Milk Products,Item_2_BEV,23.0,10.0,410.0,Digital Wallet,Online,2023-03-13,False,True


## 3. Data types

In [12]:
# Convert Transaction Date into a proper datetime object.
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'], dayfirst=True, errors='coerce')
df.head(3)

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,mismatch
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-08-04,True,False
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,NaT,True,False
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-05-10,False,False


In [13]:
#	Ensure Price Per Unit, Quantity, and Total Spent are numeric.
df.dtypes

Transaction ID              object
Customer ID                 object
Category                    object
Item                        object
Price Per Unit             float64
Quantity                   float64
Total Spent                float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
Discount Applied              bool
mismatch                      bool
dtype: object

In [14]:
# Convert Discount Applied into a Boolean (True/False).


## 4.Standardization

In [15]:
#	Standardize values in Payment Method (e.g., fix casing differences).
df['Payment Method'] = df['Payment Method'].str.strip().str.lower()
df.head(2)

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,mismatch
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,digital wallet,Online,2024-08-04,True,False
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,digital wallet,Online,NaT,True,False


In [16]:
#	Standardize Location (“Online” vs “In-store”).
#	Ensure Category values follow one naming convention
df['Location'] = df['Location'].str.strip().str.lower().replace({
    'on-line': 'online',
    'web': 'online',
    'instore': 'in-store',
    'store': 'in-store'
}).str.title()
df.head(2)

,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied,mismatch
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,digital wallet,Online,2024-08-04,True,False
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,digital wallet,Online,NaT,True,False


# Part 2: Basic Exploration (Univariate Analysis)

In [17]:
# How many total transactions are in the dataset?
total_transactions = len(df)
print("Total Transactions:", total_transactions)

Total Transactions: 12575


In [18]:
# How many unique customers are there?
df['Customer ID'].nunique()

25

In [19]:
# What is the distribution of transactions by Category?
df.groupby('Category').size()

df.groupby('Category').size() / len(df) * 100

Category
Beverages                             12.461233
Butchers                              12.469185
Computers and electric accessories    12.389662
Electric household essentials         12.652087
Food                                  12.628231
Furniture                             12.652087
Milk Products                         12.596421
Patisserie                            12.151093
dtype: float64

In [20]:
# What is the distribution of Payment Method?
df['Payment Method'].value_counts()

df['Payment Method'].value_counts(normalize=True) * 100

Payment Method
cash              34.274354
digital wallet    32.954274
credit card       32.771372
Name: proportion, dtype: float64

In [21]:
# What is the distribution of transactions by Location (Online vs In-store)?
df['Location'].value_counts()

df['Location'].value_counts(normalize=True) * 100


Location
Online      50.528827
In-Store    49.471173
Name: proportion, dtype: float64

In [22]:
# What is the overall average Total Spent per transaction?
df['Total Spent'].mean()

np.float64(128.63658051689862)

In [23]:
# What is the overall median Quantity purchased per transaction?
df['Quantity'].median()

6.0

# Part 3: Customer Analysis

In [24]:
# Which customers spent the most overall?
df.groupby('Customer ID')['Total Spent'].sum().sort_values(ascending=False)

Customer ID
CUST_24    71056.0
CUST_08    70172.5
CUST_05    70012.5
CUST_23    67979.0
CUST_13    67858.0
CUST_16    67849.0
CUST_10    65325.5
CUST_21    65320.0
CUST_15    65070.5
CUST_22    64662.0
CUST_20    64462.5
CUST_02    64325.0
CUST_19    64052.0
CUST_07    63949.5
CUST_04    63829.0
CUST_09    63702.0
CUST_12    63530.0
CUST_11    63120.0
CUST_03    62872.5
CUST_18    62722.5
CUST_14    62481.0
CUST_17    61248.0
CUST_01    61118.5
CUST_06    60911.0
CUST_25    59976.5
Name: Total Spent, dtype: float64

In [25]:
#	Which customers made the highest number of transactions
df['Customer ID'].value_counts().head()

Customer ID
CUST_05    544
CUST_24    543
CUST_13    534
CUST_08    533
CUST_09    519
Name: count, dtype: int64

In [26]:
# Which customers purchased from the most categories?

df.groupby('Customer ID')['Category'].nunique().sort_values(ascending=False).head()


Customer ID
CUST_01    8
CUST_14    8
CUST_24    8
CUST_23    8
CUST_22    8
Name: Category, dtype: int64

In [27]:
# Do customers show a preference for Online or In-store shopping?
df.groupby('Customer ID')['Location'].value_counts().unstack().fillna(0)
df['Location'].value_counts(normalize=True) * 100

Location
Online      50.528827
In-Store    49.471173
Name: proportion, dtype: float64

In [28]:
# Which customer has the largest average spend per transaction?
df.groupby('Customer ID')['Total Spent'].mean().sort_values(ascending=False).head(1)


Customer ID
CUST_03    135.209677
Name: Total Spent, dtype: float64

# Part 4: Category & Item Analysis

In [29]:
# Which category generated the most total revenue?
df.groupby('Category')['Total Spent'].sum().sort_values(ascending=False).head(1)

Category
Butchers    215930.0
Name: Total Spent, dtype: float64

In [30]:
# Which category has the highest average Total Spent per transaction?
df.groupby('Category')['Total Spent'].mean().sort_values(ascending=False).head(1)

Category
Butchers    137.710459
Name: Total Spent, dtype: float64

In [31]:
# Which items are the top sellers by total quantity sold? 
df.groupby('Item')['Quantity'].sum().sort_values(ascending=False).head()

Item
Item_2_BEV      7687.0
Item_16_MILK     627.0
Item_25_FUR      616.0
Item_19_MILK     589.0
Item_13_FOOD     581.0
Name: Quantity, dtype: float64

In [32]:
#  Are discounts applied more frequently in certain categories?
df.groupby('Category')['Discount Applied'].apply(lambda x: (x > 0).mean()).sort_values(ascending=False)


Category
Furniture                             0.694532
Beverages                             0.684110
Patisserie                            0.678010
Butchers                              0.672194
Electric household essentials         0.668133
Food                                  0.655542
Milk Products                         0.653409
Computers and electric accessories    0.649551
Name: Discount Applied, dtype: float64

In [33]:
# Which category has the highest number of unique customers buying from it?
df.groupby('Category')['Customer ID'].nunique().sort_values(ascending=False)


Category
Beverages                             25
Butchers                              25
Computers and electric accessories    25
Electric household essentials         25
Food                                  25
Furniture                             25
Milk Products                         25
Patisserie                            25
Name: Customer ID, dtype: int64

#  Part 5: Time-based Analysis

In [34]:
print("Start Date:", df['Transaction Date'].min())
print("End Date:", df['Transaction Date'].max())

Start Date: 2022-01-01 00:00:00
End Date: 2025-12-01 00:00:00


In [35]:
#	How do transactions trend over time (monthly / yearly)?
monthly_trend = df.groupby(df['Transaction Date'].dt.to_period('M')).size()
print("Monthly transaction Trend:",monthly_trend)
yearly_trend = df.groupby(df['Transaction Date'].dt.year).size()
print("Yearly transaction Trend:",yearly_trend)

Monthly transaction Trend: Transaction Date
2022-01    135
2022-02    152
2022-03    138
2022-04    125
2022-05    130
2022-06    125
2022-07    113
2022-08    136
2022-09    146
2022-10    140
2022-11    140
2022-12    141
2023-01    153
2023-02    129
2023-03    124
2023-04    116
2023-05    125
2023-06    105
2023-07    128
2023-08    122
2023-09    133
2023-10    134
2023-11    119
2023-12    134
2024-01    144
2024-02    142
2024-03    137
2024-04    136
2024-05    127
2024-06    142
2024-07    143
2024-08    161
2024-09    146
2024-10    145
2024-11    133
2024-12    144
2025-01     14
2025-02     17
2025-03     19
2025-04     14
2025-05      8
2025-06     11
2025-07      3
2025-08     20
2025-09     13
2025-10      8
2025-11     10
2025-12     13
Freq: M, dtype: int64
Yearly transaction Trend: Transaction Date
2022.0    1621
2023.0    1522
2024.0    1700
2025.0     150
dtype: int64


In [36]:
# In which month(s) did the most transactions occur?
months_with_max = df.groupby(df['Transaction Date'].dt.month).size().loc[lambda x: x == x.max()].index.tolist()
print(months_with_max)

[1.0]


In [37]:
# Which month had the highest total revenue?
df.groupby(df['Transaction Date'].dt.month)['Total Spent'].sum().idxmax()

np.float64(2.0)

In [38]:
#	Is there a difference in average transaction value between 2022, 2023, and 2024?
avg_by_year = df.groupby(df['Transaction Date'].dt.year)['Total Spent'].mean()
print(avg_by_year)

Transaction Date
2022.0    128.960827
2023.0    128.660972
2024.0    128.520882
2025.0    128.636667
Name: Total Spent, dtype: float64


In [39]:
# Do discount transactions cluster in certain months/seasons?
discount_freq_by_month = df.groupby(df['Transaction Date'].dt.month)['Discount Applied'].apply(lambda x: (x > 0).mean())
discount_freq_by_month

Transaction Date
1.0     0.686099
2.0     0.718182
3.0     0.650718
4.0     0.631714
5.0     0.646154
6.0     0.621410
7.0     0.723514
8.0     0.658314
9.0     0.643836
10.0    0.674473
11.0    0.669154
12.0    0.652778
Name: Discount Applied, dtype: float64

# Part 6: Payment & Discount Insights

In [40]:
#	Which payment method is used most often?
df['Payment Method'].mode()

0    cash
Name: Payment Method, dtype: object

In [41]:
# Which payment method generates the highest revenue?
df.groupby('Payment Method')['Total Spent'].sum().idxmax()

'cash'

In [42]:
# Does average Total Spent differ between Credit Card and Digital Wallet?
df[df['Payment Method'].isin(['Credit Card', 'Digital Wallet'])].groupby('Payment Method')['Total Spent'].mean()

Series([], Name: Total Spent, dtype: float64)

In [43]:
# What percentage of all transactions had discounts applied?
(df['Discount Applied'] > 0).mean() * 100


np.float64(66.94234592445328)

In [44]:
# Do transactions with discounts have a higher average Quantity than those without?
df.groupby(df['Discount Applied'] > 0)['Quantity'].mean()


Discount Applied
False    5.600433
True     5.538014
Name: Quantity, dtype: float64

In [45]:
# Do discount transactions generate more total revenue compared to non-discount ones?
avg_compare = df.groupby('Discount Applied')['Price Per Unit'].mean()
print(avg_compare)


Discount Applied
False    23.406663
True     23.319316
Name: Price Per Unit, dtype: float64
